In [1]:
import os
import glob
import numpy as np
import xarray as xr
import rioxarray as riox
from datetime import datetime

In [2]:
print('   setting up Local cluster...')
from dask.distributed import LocalCluster, Client
import dask
cluster = LocalCluster(n_workers=8, threads_per_worker=2)
client = Client(cluster)
display(client)

   setting up Local cluster...


2022-09-09 09:18:49,441 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-4cx3azyl', purging
2022-09-09 09:18:49,466 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-4lr1z6p5', purging
2022-09-09 09:18:49,495 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-gqdeze6u', purging
2022-09-09 09:18:49,520 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-k2ocmzga', purging
2022-09-09 09:18:49,543 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-mdrhepbl', purging
2022-09-09 09:18:49,565 - distributed.diskutils - INFO - Found stale lock file and directory '/

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39715,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:41757,Total threads: 2
Dashboard: http://127.0.0.1:43087/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:43197,


In [3]:
prefix = 'cper'
tifDIR = os.path.join('data/hls_nrt', prefix)

In [4]:
yr = 2022
da_bare = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_BARE_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_green = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_GREEN_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_sd = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_SD_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_litt = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_LITT_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_bm = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_' +
                                        str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_bm_se = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_se_' +
                                        str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_ndvi = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_ndvi',
                                        prefix + '_hls_ndvi_' +
                                        str(yr) + '.nc')).rename({'time': 'date'})#.chunk(sngl_chunks)
da_bare.name = 'BARE'
da_green.name = 'GREEN'
da_sd.name = 'SD'
da_litt.name = 'LITT'
da_bm.name = 'Biomass'
da_bm_se.name = 'Biomass_SE'
da_ndvi.name = 'NDVI'

# convert to lbs/acre
da_bm = da_bm * 0.892179
da_bm_se = np.log(np.exp(da_bm_se.chunk({'date': -1, 'y': 50, 'x': 50})  * 0.892179))

In [5]:
da_bare = da_bare.where(da_bare > 0.0, other=0.0)
da_bare = da_bare.where(da_bare < 1.0, other=1.0)

da_green = da_green.where(da_green > 0.0, other=0.0)
da_green = da_green.where(da_green < 1.0, other=1.0)

da_sd = da_sd.where(da_sd > 0.0, other=0.0)
da_sd = da_sd.where(da_sd < 1.0, other=1.0)

da_litt = da_litt.where(da_litt > 0.0, other=0.0)
da_litt = da_litt.where(da_litt < 1.0, other=1.0)

cov_mask = da_bare.notnull() & da_green.notnull() & da_sd.notnull() & da_litt.notnull()

In [6]:
ds = xr.merge([da_bare.where(cov_mask), 
               da_green.where(cov_mask), 
               da_sd.where(cov_mask),
               da_litt.where(cov_mask),
               da_bm.where((da_bm >= 0) & (da_bm < 5000)),
               da_bm_se.where((da_bm >= 0) & (da_bm < 5000)),
               da_ndvi.where((da_ndvi >= 0) & (da_ndvi <= 1))])
ds = ds.where(ds != -9999.)

In [7]:
cov_sum = ds['BARE']+ds['SD']+ds['LITT']+ds['GREEN'].compute()
#cov_sum

In [8]:
ds['BARE'] = ds['BARE'] / cov_sum
ds['SD'] = ds['SD'] / cov_sum
ds['LITT'] = ds['LITT'] / cov_sum
ds['GREEN'] = ds['GREEN'] / cov_sum

In [9]:
#ds['date'] = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in ds['date'].values]

In [10]:
#ds['date'] = ds['date'].dt.date
ds['DRYVEG'] = ds['SD'] + ds['LITT']
ds['NONVEG'] = ds['BARE'] + ds['LITT']

In [11]:
ds.rio.crs

CRS.from_epsg(3857)

In [12]:
#ds.rio.set_crs(3857)

In [13]:
ds.rio.reproject("EPSG:3857").to_netcdf('data/gcloud/hls_cper_' + str(yr) + '_gcloud.nc')

/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*(_execute_task(a, cache) for a in args))
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*(_execute_task(a, cache) for a in args))
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*(_execute_task(a, cache) for a in args))
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*(_execute_task(a, cache) for a in args))
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in exp
  return func(*(_execute_task(a, cache) for a in args))
/home/spkearney/miniconda3/envs/hls_nrt_env/l